In [126]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from shapely.geometry import *
root = ET.parse('mmi-mmi_india (4).xml')


In [127]:
namespace = {"georss": "http://www.georss.org/georss"}
items = root.findall('.//item', namespace)

lines = []
points =[]

geometry_tags = {
    '{http://www.georss.org/georss}line' : lambda x: LineString([(eval(x[i+1]), eval(x[i])) for i in range(0, len(x)-1, 2)]),
    '{http://www.georss.org/georss}point' : lambda x: Point(eval(x[1]), eval(x[0])),
}

for item in items:
        # Extract main data
    
    parsed_data = {
        "title": item.find("title").text,
        # "link": item.find("link").text.strip(),
        # "guid": item.find("guid").text.strip()
    }

    # Parse <description> section (HTML content)
    description_html = item.find("description").text
    soup = BeautifulSoup(description_html, "html.parser")

    # Extract fields from description
    attributes = {}
    for li in soup.find_all("li"):
        key = li.find("span", {"class": "atr-name"})
        value = li.find("span", {"class": "atr-value"})
        if key and value:
            attributes[key.text.strip()] = value.text.strip()

    parsed_data.update(attributes)
    
    geometry = item.find("georss:*", namespace).text
    tag = (item.find("georss:*", namespace).tag)
    fn = geometry_tags.get(tag)
    
    if gf: 
        parsed_data['geometry'] = fn(geometry.split(' '))
    
    if isinstance(parsed_data['geometry'], Point):
        points.append(parsed_data)
    
    if isinstance(parsed_data['geometry'], LineString):
        lines.append(parsed_data)

In [128]:
import geopandas as gpd

In [129]:
poidf = gpd.GeoDataFrame(points)
poidf['title'] = poidf.title.apply(lambda x : x.split('.')[0])

In [130]:
poidf.loc[poidf.title == 'ka_poi_q4_2022',['title','geometry','id','poi_nme','ftr_cry','cat_dec','loc_nme','city_nme','stt_nme','dist_nme']].set_geometry('geometry').set_crs('EPSG:4326').to_file('data1.gpkg', layer = 'POIS')
gpd.GeoDataFrame(lines).set_geometry('geometry').set_crs('EPSG:4326').to_file('data1.gpkg', layer = 'ROADS')